In [5]:
import sys, os
import numpy as np
np.set_printoptions(threshold=np.inf)
import pandas as pd
from utils.utils import read_txt, read_txt_np_int
from functools import lru_cache
from math import ceil, floor
from copy import deepcopy

# INPUT

In [6]:
inputfilename = './inputs/day21.txt'

inputdata = read_txt(inputfilename)

testdata_small = ["029A",
"980A",
"179A",
"456A",
"379A"]

code_keyboard = np.array([['7', '8', '9'],['4', '5', '6'],['1', '2', '3'],['', '0', 'A']])

direction_keyboard = np.array([['', '^', 'A'],['<', 'v', '>']])

In [93]:
data_to_use = testdata_small

movement_map = {'<': np.array([0, -1]), 'v': np.array([1, 0]), '>': np.array([0, 1]), '^': np.array([-1, 0])}

keyboard_chain = ['movement', 'movement', 'movement', 'code']

data_to_use

['029A', '980A', '179A', '456A', '379A']

## PART 1

In [102]:
class Robot:

    def __init__(self, robot_dict, keyboard):
        self.dict = robot_dict
        self.keyboard = keyboard
        self.sign = 'A'

    # Get the list of transitions from a code
    @lru_cache()
    def process_code_reverse(self, code, sign = 'A'):
        current_sign = sign
        transition_movements = []
        next_sign = code[0]
        transition = current_sign + next_sign
        transition_movements.append(self.dict[transition])
        if len(code) <= 1:
            return transition_movements
        transition_movements += self.process_code_reverse(code[1:], next_sign)

        # Compact list
        pending_movement = ''
        output_movements = []

        for movement_list in transition_movements:
            if len(movement_list) == 1:
                pending_movement += movement_list[0]
                continue
            if len(pending_movement) > 0:
                output_movements.append([pending_movement])
            output_movements.append(movement_list)
            pending_movement = ''
        
        if len(pending_movement) > 0:
            output_movements.append([pending_movement])

        return output_movements
    
    def process_code(self, code, sign = 'A'):
        position = (int(np.where(self.keyboard == sign)[0][0]), int(np.where(self.keyboard == sign)[1][0]))
        output_values = []
        for char in code:
            if char == 'A':
                output_values += self.keyboard[position[0], position[1]]
            else:
                movement = movement_map[char]
                position = (position[0]+movement[0], position[1]+movement[1])
        return ''.join(output_values)


def all_orderings(movement_string):
    if len(movement_string) <= 1:
        return set(movement_string)
    orderings = set()
    for index in range(len(movement_string)):
        for next_ordering in all_orderings(movement_string[:index] + movement_string[index+1:]):
            orderings.add(movement_string[index] + next_ordering)
    return orderings

def get_all_paths(starting_position, final_position):
    vertical_delta = final_position[0] - starting_position[0]
    horizontal_delta = final_position[1] - starting_position[1]
    movement_string = ''
    vertical_sign, horizontal_sign = '', ''
    if vertical_delta > 0:
        vertical_sign = 'v'
    if vertical_delta < 0:
        vertical_sign = '^'
    if horizontal_delta > 0:
        horizontal_sign = '>'
    if horizontal_delta < 0:
        horizontal_sign = '<'

    movement_string += abs(vertical_delta)*vertical_sign + abs(horizontal_delta)*horizontal_sign

    # Each transition has to finish by pressing A
    all_movements = [movement + 'A' for movement in all_orderings(movement_string)]

    if horizontal_delta == vertical_delta == 0:
        all_movements = ['A']

    return all_movements

def valid_path(keyboard, starting_position, path):
    current_position = starting_position
    for direction in path:
        if direction == 'A':
            continue
        movement = movement_map[direction]
        current_position = (current_position[0] + movement[0], current_position[1]+movement[1])
        # Panic check
        #print(f"Visited position: {current_position}")
        if keyboard[current_position[0],current_position[1]] == '':
            return False
    return True

def process_keyboard(keyboard):
    keyboard_dict = {}
    for starting_sign in keyboard.flatten():
        if starting_sign == '':
            continue
        for ending_sign in keyboard.flatten():
            if ending_sign == '':
                continue

            # Only transition to a different sign (why?)
            if ending_sign == starting_sign:
                pass        # let's keep all transitions
            starting_position = (int(np.where(keyboard == starting_sign)[0][0]), int(np.where(keyboard == starting_sign)[1][0]))
            ending_position = (int(np.where(keyboard == ending_sign)[0][0]), int(np.where(keyboard == ending_sign)[1][0]))
            transition = starting_sign + ending_sign

            tentative_paths = get_all_paths(starting_position, ending_position)
            keyboard_dict[transition] = [path for path in tentative_paths if valid_path(keyboard, starting_position, path)]
    return keyboard_dict

# Each combination to be entered requires an 'A' after each symbol -- only for movement
def expand_code_list(codelist, minimize = False):
    expanded_code_list = ['']
    for codeset in codelist:
        next_expanded_code_list = []
        if minimize:
            min_len = 1000000
            for code in codeset:
                if len(code) < min_len:
                    shortest_code = code
            expanded_code_list = [expanded_code_list[0] + shortest_code]
        else:
            for code in codeset:
                for previous_code in expanded_code_list:
                    next_expanded_code_list.append(previous_code + code)
            expanded_code_list = next_expanded_code_list
    return expanded_code_list

'''
def process_code(code, keyboard_dict, starting_sign):
    current_sign = starting_sign
    transition_movements = []
    for next_sign in code:
        transition = current_sign + next_sign
        #print(transition)
        current_sign = next_sign
        transition_movements.append(keyboard_dict[transition])
    return transition_movements
'''


######################################################################################################################

direction_keyboard_dict = process_keyboard(direction_keyboard)
code_keyboard_dict = process_keyboard(code_keyboard)

keyboard_config = {'movement': direction_keyboard_dict, 'code': code_keyboard_dict}

robot_code = Robot(keyboard_config['code'], code_keyboard) # Returns input for previous

robot_movement_1 = Robot(keyboard_config['movement'], direction_keyboard) # Returns input for previous
robot_movement_2 = Robot(keyboard_config['movement'], direction_keyboard) # Returns input for previous


In [103]:
keyboard_config['movement']

{'^^': ['A'],
 '^A': ['>A'],
 '^<': ['v<A'],
 '^v': ['vA'],
 '^>': ['v>A', '>vA'],
 'A^': ['<A'],
 'AA': ['A'],
 'A<': ['v<<A', '<v<A'],
 'Av': ['v<A', '<vA'],
 'A>': ['vA'],
 '<^': ['>^A'],
 '<A': ['>^>A', '>>^A'],
 '<<': ['A'],
 '<v': ['>A'],
 '<>': ['>>A'],
 'v^': ['^A'],
 'vA': ['^>A', '>^A'],
 'v<': ['<A'],
 'vv': ['A'],
 'v>': ['>A'],
 '>^': ['<^A', '^<A'],
 '>A': ['^A'],
 '><': ['<<A'],
 '>v': ['<A'],
 '>>': ['A']}

In [104]:
robot_code.process_code_reverse('029A', keyboard_config['code'], 'A')

TypeError: unhashable type: 'dict'

In [105]:
cumulator = 0


for entry_code in data_to_use:

    print(entry_code)

    starting_symbol = 'A'

    code_list = [set([entry_code])]
    expanded_code_list = expand_code_list(code_list)

    keyboard = 'code'

    print(f"Processing keyboard {keyboard}")
    print(f"Expanded code list: {expanded_code_list}")
    previous_code_list = []
    for code in expanded_code_list:
        print(code)
        print(robot_code.process_code_reverse(code, starting_symbol))
        previous_code_list += expand_code_list(robot_code.process_code_reverse(code, starting_symbol))

    print(f"Previous code list: {previous_code_list}") # movement 1

    #######################################################################################################################

    expanded_code_list = set(previous_code_list)
    keyboard = 'movement'

    print(f"Processing keyboard {keyboard}")
    print(f"Expanded code list: {expanded_code_list}")
    previous_code_list = []
    for code in expanded_code_list:
        previous_code_list += expand_code_list(robot_movement_1.process_code_reverse(code, starting_symbol))

    print(f"Previous code list: {previous_code_list}") # movement 2

    #######################################################################################################################

    expanded_code_list = set(previous_code_list)
    keyboard = 'movement'

    print(f"Processing keyboard {keyboard}")
    print(f"Expanded code list: {expanded_code_list}")
    previous_code_list = []
    for code in expanded_code_list:
        previous_code_list += expand_code_list(robot_movement_2.process_code_reverse(code, starting_symbol), minimize = True)

    print(f"Previous code list: {previous_code_list}") # movement 3

    min_len = 1000000
    for code in previous_code_list:
        if len(code) < min_len:
            min_len = len(code)
            shortest = code

    print(f"Shortest code: {shortest}, len {min_len}")

    cumulator += min_len * int(entry_code[:-1])

print(cumulator)


029A
Processing keyboard code
Expanded code list: ['029A']
029A
[['<A^A'], ['^^>A', '^>^A', '>^^A'], ['vvvA']]
Previous code list: ['<A^A^^>AvvvA', '<A^A^>^AvvvA', '<A^A>^^AvvvA']
Processing keyboard movement
Expanded code list: {'<A^A>^^AvvvA', '<A^A^>^AvvvA', '<A^A^^>AvvvA'}
Previous code list: ['v<<A>^>A<A>AvA<^AA>Av<AAA^>A', '<v<A>^>A<A>AvA<^AA>Av<AAA^>A', 'v<<A>>^A<A>AvA<^AA>Av<AAA^>A', '<v<A>>^A<A>AvA<^AA>Av<AAA^>A', 'v<<A>^>A<A>AvA^<AA>Av<AAA^>A', '<v<A>^>A<A>AvA^<AA>Av<AAA^>A', 'v<<A>>^A<A>AvA^<AA>Av<AAA^>A', '<v<A>>^A<A>AvA^<AA>Av<AAA^>A', 'v<<A>^>A<A>AvA<^AA>A<vAAA^>A', '<v<A>^>A<A>AvA<^AA>A<vAAA^>A', 'v<<A>>^A<A>AvA<^AA>A<vAAA^>A', '<v<A>>^A<A>AvA<^AA>A<vAAA^>A', 'v<<A>^>A<A>AvA^<AA>A<vAAA^>A', '<v<A>^>A<A>AvA^<AA>A<vAAA^>A', 'v<<A>>^A<A>AvA^<AA>A<vAAA^>A', '<v<A>>^A<A>AvA^<AA>A<vAAA^>A', 'v<<A>^>A<A>AvA<^AA>Av<AAA>^A', '<v<A>^>A<A>AvA<^AA>Av<AAA>^A', 'v<<A>>^A<A>AvA<^AA>Av<AAA>^A', '<v<A>>^A<A>AvA<^AA>Av<AAA>^A', 'v<<A>^>A<A>AvA^<AA>Av<AAA>^A', '<v<A>^>A<A>AvA^<AA>Av<AAA>^A

In [106]:
command_to_test = '<vA<AA^>>AA<A>vA^AvA^A<<vA^>>AAvA^A<vA>^AA<A>A<<vA>A>^AAA<A>vA^A'
print(f"Command lenght {len(command_to_test)}")
first_output = robot_movement_2.process_code(command_to_test, 'A')
print(first_output)
second_output = robot_movement_1.process_code(first_output, 'A')
print(second_output)
final_output = robot_code.process_code(second_output, 'A')
print(final_output)

Command lenght 64
v<<AA^>A>A<AA>AvAA^A<vAAA^>A
<<^A^^A>>AvvvA
179A


## PART 2

In [107]:
def send_code(code, num_dir_robots):
    for next in range(num_dir_robots):
        code = robot_movement_1.process_code(code, 'A')
    code = robot_code.process_code(code, 'A')
    print(code)

In [108]:
send_code('<vA<AA>>^AvAA<^A>A<v<A>>^AvA^A<vA>^A<v<A>^A>AAvA^A<v<A>A>^AAAvA<^A>A', 2)

029A


In [109]:
robot_movement_1.process_code_reverse('<vA<AA>>^AvAA<^A>A<v<A>>^AvA^A<vA>^A<v<A>^A>AAvA^A<v<A>A>^AAAvA<^A>A')

[['v<<A', '<v<A'],
 ['>A'],
 ['^>A', '>^A'],
 ['v<<A', '<v<A'],
 ['>^>A', '>>^A'],
 ['AvAA'],
 ['<^A', '^<A'],
 ['>A'],
 ['v<A', '<vA'],
 ['^>A', '>^A'],
 ['A'],
 ['v<<A', '<v<A'],
 ['>^A>AvA^A'],
 ['v<<A', '<v<A'],
 ['>A<A'],
 ['>^>A', '>>^A'],
 ['vAA'],
 ['<^A', '^<A'],
 ['>A'],
 ['v<A', '<vA'],
 ['^>A', '>^A'],
 ['<A>A'],
 ['v<<A', '<v<A'],
 ['>A'],
 ['^>A', '>^A'],
 ['vA'],
 ['<^A', '^<A'],
 ['>A'],
 ['v<<A', '<v<A'],
 ['>A<A'],
 ['>^>A', '>>^A'],
 ['vA'],
 ['<^A', '^<A'],
 ['>AvA^AA'],
 ['v<A', '<vA'],
 ['^>A', '>^A'],
 ['<A>A'],
 ['v<<A', '<v<A'],
 ['>A<A'],
 ['>^>A', '>>^A'],
 ['vA^AvA'],
 ['<^A', '^<A'],
 ['>AAA'],
 ['v<A', '<vA'],
 ['^>A', '>^A'],
 ['v<<A', '<v<A'],
 ['>^A>AvA^A']]